In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
spark = SparkSession.builder.appName("h1agg").config("spark.driver.host", "localhost").getOrCreate()
conf = spark.sparkContext._conf.setAll([('spark.driver.memory', '12g')])

In [ ]:
h1=spark.read.parquet("/home/jovyan/dataset/h1_parquet.gz.parquet").repartition(1000)

In [ ]:
h1.groupBy(F.col("INDEX")).agg(F.mean("GM")).count()

In [ ]:
spark.sparkContext.getConf().getAll()

In [ ]:
# TODO:
# 1. add column with lag
# 2. mean row wise 
# ref: https://stackoverflow.com/questions/32670958/spark-dataframe-computing-row-wise-mean-or-any-aggregate-operation
        
# 3. rank by index -> top N
# 4. mean(abs(diff)) , mean(diff) groupBy index
# 5. 50-150 : best lookback vs. 151-220 best lookback at each weight
    
    

In [ ]:
def select_top_ranker(sdf, groupByCol, orderByCol):
    rankWindow = Window.partitionBy(groupByCol).orderBy(F.col(orderByCol).desc())
    sdf = sdf.withColumn("RANK", F.rank().over(rankWindow))
    return sdf

In [ ]:
def row_mean(sdf, mean_col_str, lookback_max):
    mean_target_cols = list(map(lambda  x: mean_col_str+str(x), range(1,lookback_max+1)))
    # range excludes end, that's why +1
    print(mean_target_cols)
    mean_col=mean_col_str+"_mean_"+str(lookback_max)
    sdf=sdf.withColumn(mean_col, (sum(F.col(x) for x in mean_target_cols)/F.lit(lookback_max)))
    return sdf

In [ ]:
def lag_col(sdf, groupBy_col_str,lag_col_str, lag_step):
    sdf=(sdf
         .withColumn(lag_col_str+str(lag_step),
                     F.lag(sdf[lag_col_str],lag_step).over(Window
                                                  .partitionBy(groupBy_col_str)
                                                  .orderBy(["KEY","INDEX"]))))
    return sdf


In [ ]:
mean_weight=0.7

In [ ]:
h1=h1.withColumn("KEY",F.concat(F.col("rank_bin"),F.lit("_"),F.col("FEATURE_BIN")))
h1=h1.withColumn("WG", F.col("GM")*mean_weight+F.col("GMD")*(1-mean_weight))

In [ ]:
lookbackdays_list = range(1,50+1)


In [ ]:
for lbd in lookbackdays_list:
    h1 = lag_col(h1,"KEY","WG",lbd)

In [ ]:
h1=row_mean(h1,"WG",20)

In [ ]:
h1=h1.where("WG_mean_20 is not null")
h1=select_top_ranker(h1, "INDEX","WG_mean_20")

In [ ]:
h1_top5 = h1.where("rank <= 5")
h1_agg = h1_top5.groupBy("INDEX").agg(F.mean("WG").alias("WG_MEAN")).orderBy("INDEX")

In [ ]:
h1_agg.limit(10).head()

In [ ]:
spark.sparkContext.uiWebUrl

In [ ]:
l=spark.sparkContext

In [ ]:
l.getConf().getAll()

In [ ]:
h1.limit(10).toPandas()